# Analyse de la Stratégie Long-Short basée sur le Momentum

Ce notebook analyse et visualise les résultats de notre stratégie de trading long-short basée sur le momentum. La stratégie prend des positions longues sur les actions ayant le momentum le plus faible et des positions courtes sur les actions ayant le momentum le plus élevé, suivant le principe de réversion à la moyenne.

## 1. Configuration et chargement des modules

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime, timedelta

# Configuration pour les visualisations
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 8)
plt.rcParams['font.size'] = 12

# Ajout du dossier parent au chemin pour pouvoir importer les modules
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

In [ ]:
# Import des modules de notre plateforme
from momentum_strategy import MomentumStrategy
from data_loader import DataLoader
from backtest import Backtest
from visualization import plot_strategy_performance, plot_position_evolution, plot_momentum_heat_map, plot_correlation_matrix

## 2. Définition des paramètres de la stratégie

In [ ]:
# Liste des actifs mentionnés dans le document
assets = ['BA', 'PEP', 'LMT', 'AAPL', 'JNJ', 'JPM', 'BAC', 'LLY', 'CAR']

# Période d'analyse
start_date = '2024-12-01'
end_date = '2025-04-30'

# Paramètres de la stratégie
momentum_period = 30  # Période pour le calcul du momentum (en jours)
forward_return_days = 5  # Jours pour calculer les rendements futurs
quantiles = (0.2, 0.8)  # Quantiles pour les positions longues (20%) et courtes (80%)

## 3. Téléchargement et préparation des données

In [ ]:
# Création de l'instance de la stratégie
strategy = MomentumStrategy(assets)

# Téléchargement des données
data = strategy.download_data(start_date, end_date)

# Affichage des premières lignes des données
data.head()

In [ ]:
# Visualisation de l'évolution des prix
normalized_prices = data / data.iloc[0]
plt.figure(figsize=(15, 8))
normalized_prices.plot()
plt.title('Évolution des Prix Normalisés')
plt.ylabel('Prix (normalisé à 1 au début)')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid(True)
plt.show()

## 4. Calcul du momentum et des rendements futurs

In [ ]:
# Calcul du momentum
momentum_scores = strategy.calculate_momentum(momentum_period)

# Affichage des scores de momentum récents
momentum_scores.tail()

In [ ]:
# Calcul des rendements futurs
returns = strategy.calculate_forward_returns(forward_return_days)

# Affichage des rendements futurs récents
returns.tail()

In [ ]:
# Visualisation des scores de momentum sous forme de heatmap
heatmap_fig = plot_momentum_heat_map(momentum_scores, window=10, figsize=(15, 8))
plt.show()

## 5. Analyse de la corrélation entre momentum et rendements futurs

In [ ]:
# Calcul des corrélations
correlations = strategy.calculate_correlations()
correlations

In [ ]:
# Visualisation des corrélations
plt.figure(figsize=(15, 7))
plt.bar(range(len(correlations)), correlations['Correlation'])
plt.xticks(range(len(correlations)), correlations.index, rotation=90)
plt.axhline(0, color='black', linestyle='--')
plt.title('Corrélations Momentum-Rendements par Action')
plt.ylabel('Corrélation de Spearman')
plt.tight_layout()
plt.show()

In [ ]:
# Visualisation de la matrice de corrélation des rendements
corr_fig = plot_correlation_matrix(returns)
plt.show()

## 6. Mise en œuvre de la stratégie long-short

In [ ]:
# Classement des actifs
ranks = strategy.rank_assets()

# Affichage des rangs récents
ranks.tail()

In [ ]:
# Création du portefeuille long-short
long_portfolio, short_portfolio = strategy.create_long_short_portfolio(ranks, quantiles)

# Affichage des positions longues récentes
print("Positions longues récentes:")
long_portfolio.tail()

In [ ]:
# Affichage des positions courtes récentes
print("Positions courtes récentes:")
short_portfolio.tail()

In [ ]:
# Visualisation de l'évolution des positions
pos_fig = plot_position_evolution(long_portfolio, short_portfolio)
plt.show()

## 7. Calcul et analyse des rendements de la stratégie

In [ ]:
# Calcul des rendements de la stratégie
strategy_returns = strategy.calculate_strategy_returns(long_portfolio, short_portfolio)

# Affichage des rendements récents
strategy_returns.tail()

In [ ]:
# Visualisation des rendements cumulatifs
plt.figure(figsize=(15, 7))
cumulative_returns = (1 + strategy_returns).cumprod()
cumulative_returns.plot()
plt.title('Rendements Cumulatifs de la Stratégie')
plt.ylabel('Rendements Cumulatifs')
plt.grid(True)
plt.show()

In [ ]:
# Statistiques descriptives des rendements
print("Statistiques descriptives des rendements:")
print(strategy_returns.describe())

# Calcul des métriques de performance
total_return = cumulative_returns.iloc[-1] - 1
annualized_return = (1 + total_return) ** (252 / len(strategy_returns)) - 1
volatility = strategy_returns.std() * np.sqrt(252)
sharpe_ratio = annualized_return / volatility if volatility != 0 else 0
max_drawdown = (cumulative_returns / cumulative_returns.cummax() - 1).min()
win_rate = (strategy_returns > 0).mean()

print("\nMétriques de performance:")
print(f"Rendement total: {total_return:.2%}")
print(f"Rendement annualisé: {annualized_return:.2%}")
print(f"Volatilité annualisée: {volatility:.2%}")
print(f"Ratio de Sharpe: {sharpe_ratio:.2f}")
print(f"Drawdown maximum: {max_drawdown:.2%}")
print(f"Taux de réussite: {win_rate:.2%}")

In [ ]:
# Distribution des rendements
plt.figure(figsize=(15, 7))
sns.histplot(strategy_returns, kde=True, bins=30)
plt.axvline(0, color='black', linestyle='--')
plt.title('Distribution des Rendements de la Stratégie')
plt.xlabel('Rendement')
plt.ylabel('Fréquence')
plt.grid(True)
plt.show()

## 8. Backtest complet avec benchmark

In [ ]:
# Exécution d'un backtest complet
backtest = Backtest(strategy, start_date, end_date)
results = backtest.run(
    momentum_period=momentum_period,
    forward_return_days=forward_return_days,
    quantiles=quantiles,
    transaction_cost=0.001
)

In [ ]:
# Affichage du résumé des performances
backtest.print_performance_summary()

In [ ]:
# Visualisation des performances par rapport au benchmark
perf_fig = backtest.plot_performance(figsize=(15, 12))
plt.show()

In [ ]:
# Visualisation de l'évolution du portefeuille
port_fig = backtest.plot_portfolio_evolution()
plt.show()

## 9. Analyse de sensibilité aux paramètres

In [ ]:
# Test de différentes périodes de momentum
momentum_periods = [5, 10, 20, 30, 60, 90]
momentum_results = {}

for period in momentum_periods:
    strategy.calculate_momentum(period)
    strategy.calculate_forward_returns(5)
    ranks = strategy.rank_assets()
    long_portfolio, short_portfolio = strategy.create_long_short_portfolio(ranks)
    returns = strategy.calculate_strategy_returns(long_portfolio, short_portfolio)
    momentum_results[period] = returns

In [ ]:
# Comparaison des rendements cumulatifs selon la période de momentum
plt.figure(figsize=(15, 7))
cum_returns = pd.DataFrame({
    f'Momentum {period} jours': (1 + rets).cumprod() for period, rets in momentum_results.items()
})
cum_returns.plot()
plt.title('Rendements Cumulatifs selon la Période de Momentum')
plt.ylabel('Rendements Cumulatifs')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Métriques de performance pour chaque période
performance_metrics = pd.DataFrame(index=momentum_periods, columns=['Total Return', 'Sharpe Ratio', 'Max Drawdown', 'Win Rate'])

for period in momentum_periods:
    returns = momentum_results[period]
    cum_returns = (1 + returns).cumprod()
    total_return = cum_returns.iloc[-1] - 1
    volatility = returns.std() * np.sqrt(252)
    sharpe_ratio = (returns.mean() * 252) / volatility if volatility != 0 else 0
    max_drawdown = (cum_returns / cum_returns.cummax() - 1).min()
    win_rate = (returns > 0).mean()
    
    performance_metrics.loc[period] = [total_return, sharpe_ratio, max_drawdown, win_rate]

performance_metrics

In [ ]:
# Visualisation des métriques de performance
plt.figure(figsize=(15, 10))

# Subplot pour chaque métrique
plt.subplot(2, 2, 1)
performance_metrics['Total Return'].plot(kind='bar')
plt.title('Rendement Total')
plt.grid(True, axis='y')

plt.subplot(2, 2, 2)
performance_metrics['Sharpe Ratio'].plot(kind='bar')
plt.title('Ratio de Sharpe')
plt.grid(True, axis='y')

plt.subplot(2, 2, 3)
performance_metrics['Max Drawdown'].plot(kind='bar')
plt.title('Drawdown Maximum')
plt.grid(True, axis='y')

plt.subplot(2, 2, 4)
performance_metrics['Win Rate'].plot(kind='bar')
plt.title('Taux de Réussite')
plt.grid(True, axis='y')

plt.tight_layout()
plt.show()

## 10. Test sur différents univers d'actions

In [ ]:
# Création de différents portfolios
portfolios = {
    'Original': ['BA', 'PEP', 'LMT', 'AAPL', 'JNJ', 'JPM', 'BAC', 'LLY', 'CAR'],
    'Tech': ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA', 'TSLA', 'ADBE', 'CRM'],
    'Finance': ['JPM', 'BAC', 'WFC', 'C', 'GS', 'MS', 'BLK', 'AXP', 'V'],
    'Healthcare': ['JNJ', 'PFE', 'UNH', 'MRK', 'ABBV', 'TMO', 'ABT', 'DHR', 'BMY']
}

portfolio_results = {}

for name, stocks in portfolios.items():
    print(f"\nTest sur le portfolio {name}...")
    strat = MomentumStrategy(stocks)
    try:
        results = strat.backtest('2024-12-01', '2025-04-30', 30, 5)
        portfolio_results[name] = results['strategy_returns']
        
        total_return = results['cumulative_returns'].iloc[-1] - 1
        print(f"Rendement total: {total_return:.2%}")
    except Exception as e:
        print(f"Erreur: {str(e)}")

In [ ]:
# Comparaison des rendements cumulatifs
plt.figure(figsize=(15, 7))
cum_returns = pd.DataFrame({
    name: (1 + rets).cumprod() for name, rets in portfolio_results.items() if not rets.empty
})
cum_returns.plot()
plt.title('Rendements Cumulatifs par Type de Portfolio')
plt.ylabel('Rendements Cumulatifs')
plt.grid(True)
plt.legend()
plt.show()

## 11. Conclusion et observations

### Observations clés

1. **Corrélation Momentum-Rendements :** 
   - La stratégie repose sur l'hypothèse d'une corrélation négative entre le momentum et les rendements futurs (réversion à la moyenne).
   - Nous avons observé que cette corrélation varie selon les actifs, certains montrant une corrélation plus forte que d'autres.

2. **Performance de la Stratégie :**
   - La stratégie a généré un rendement total de [calculé dans le notebook] sur la période.
   - Le ratio de Sharpe de [calculé dans le notebook] indique un bon rapport rendement/risque.

3. **Sensibilité aux Paramètres :**
   - La période de calcul du momentum a un impact significatif sur les résultats.
   - Une période de [identifiée dans le notebook] jours semble optimale pour notre univers d'actions.

4. **Comparaison par Secteur :**
   - La stratégie performe différemment selon les secteurs.
   - Le secteur [identifié dans le notebook] a montré les meilleurs résultats, ce qui suggère que la réversion à la moyenne est plus prononcée dans ce secteur.

### Améliorations possibles

1. **Filtrage des Actions :**
   - Intégrer un filtre de volatilité pour sélectionner les actions avant de les classer par momentum.
   - Exclure les actions avec des événements spécifiques (annonces de résultats, fusions, etc.).

2. **Pondération Dynamique :**
   - Ajuster la pondération des positions en fonction de la force du signal de momentum.
   - Intégrer d'autres facteurs comme la volatilité ou la liquidité dans le modèle de pondération.

3. **Timing d'Entrée/Sortie :**
   - Optimiser la fréquence de rebalancement du portefeuille.
   - Intégrer des indicateurs techniques pour améliorer le timing des entrées et sorties.

4. **Multi-facteur :**
   - Combiner le momentum avec d'autres facteurs comme la valeur, la qualité ou la taille.
   - Développer un modèle de scoring composite pour la sélection des actions.